In [8]:
def youtube_audio_downloader(link):
    from pytube import YouTube
    import os
    import re
    
    if 'youtube.com' not in link:
        print('Invalid Youtube link!')
        return false
    try:
        yt = YouTube(link)
    except Exception as e:
        print(f'Error when creating the Youtube instance :{e}')
    
    audio = yt.streams.filter(only_audio=True).first()
    print('Downloading the audio stream...', end='')
    output_file = audio.download()
    if os.path.exists(output_file):
        print('Done!')
    else:
        print('Error downloading rhe audio file')
        return false
    
    basename = os.path.basename(output_file)
    name, extension = os.path.splitext(basename)
    audio_file = f'{name}.mp3'
    audio_file = re.sub('\s+','-',audio_file)
    os.rename(basename,audio_file)
    return audio_file

def transcribe(audio_file):
    import os
    import openai
    
    if not os.path.exists(audio_file):
        print('Audio file does not exist!')
        return false
    
    with open(audio_file, 'rb') as f:
        print('Starting transcribing...', end='')
        transcript = openai.Audio.transcribe('whisper-1',f)
        print('Done!')
        
        name, extension = os.path.splitext(audio_file)
        transcript_filename = f'transcript-{name}.txt'
        with open(transcript_filename, 'w') as f:
            f.write(transcript['text'])
        
    return transcript_filename

def summarize(transcript_filename):
    import openai
    import os
    
    if not os.path.exists(transcript_filename):
        print('The transcript file does not exists!')
        return false
    with open(transcript_filename) as f:
        transcript = f.read()
        
    system_prompt ='I want you to act as a Life Coach. Translate into Spanish'
    prompt = f'''Create a summary of the following text.
    Text: {transcript}
    
    Add a title to the summary.
    Your summary should be informative and factual, covering the most important aspect of the topic
    Start your summary with an INTRODUCTION PARAGRAPH that gives an overview of the topic FOLLOWED
    by BULLET POINTS if possible AND end the summary with a CONCLUSION PHRASE.
    '''
    print('Starting sumarizing...', end='')
    
    response = openai.ChatCompletion.create(
        model='gpt-3.5-turbo',
        messages=[
            {'role':'system', 'content':system_prompt},
            {'role':'user', 'content':prompt}
        ],
        max_tokens=2048,
        temperature=1
    )
    print('Done')
    r = response['choices'][0]['message']['content']
    return r
    

In [10]:
import openai
from dotenv import dotenv_values
config = dotenv_values(".env")
openai.api_key = config['OPEN_API_KEY']
assert openai.api_key.startswith('sk-'), 'Error loading the API key. The API key should start with "sk-""'


link = 'https://www.youtube.com/watch?v=x6kgaPqF4g4'
mp3_file = youtube_audio_downloader(link)
transcript_file = transcribe(mp3_file)
summary = summarize(transcript_file)
print('\n')
print(summary)

Starting transcribing...Done!
Starting sumarizing...

InvalidRequestError: This model's maximum context length is 4097 tokens. However, you requested 4476 tokens (2428 in the messages, 2048 in the completion). Please reduce the length of the messages or completion.